# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "4")
         .config("spark.executor.memory", "5g")
         .config("spark.executor.cores", "6")
         .config("spark.driver.memory", "5g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "10g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/09/30 06:50:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [5]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.filtrar_sentencias(preprocess=solo_portanto)

estructurales = SentenciasEstructurales(seleccion)

# Formar dataset de sentencias estructurales

In [6]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

21/09/30 06:51:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Cargar procesamiento de sentencias estructurales.

In [7]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
s = estructurales.seleccion.sdf
s.limit(10).toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,0,"[Marjorie Carmona Solano, Elías Villegas Matar...","[Circuito 02 de la, Colegio Líder INVU Las Cañas]","[Directora de la Escuela, Dirección Regional d...","[Director Regional de Educación de Alajuela, I]",None,"[Instituto Nacional de Vivienda y Urbanismo, I...",24112,2582,None
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,1,"[Hazel Córdoba Soto, Jefe del Área de Servicio...","[Ministerio de Educación Pública, Ministro de ...",None,None,None,[Ministerio de Educacion Publica],6022,1394,[1970-01-03 00:00:00]
2,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. N...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,...,1,"[Karla González, Heredia-San José, Ministra]","[Ministra de Obras Publicas y Transportes, Min...",None,[Dependencias Técnicas],None,None,19759,1575,[1970-04-01 00:00:00]
3,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. N...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,1,"[Jeremías Vargas Chavarría, Carmen Rivera Rivera]",[Ministerio de Trabajo y Seguridad],[Director Nacional de Pensiones],None,None,None,14270,1466,None
4,715B6.html,*050147230007CO* Exp: 05-014723-0007-CO Res. N...,PETICION,0.273704,050147230007CO,2006-02-10 11:51:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,0,"[Sonia Montero Díaz, Walter Betrano Valverde, ...",[Alcaldesa Municipal de Montes de Oca],None,None,None,None,9301,1275,None
5,6FF2A.html,*050143170007CO* Exp: 05-014317-0007-CO Res. N...,PENSION,0.680464,050143170007CO,2006-01-27 12:25:00,Molina Quesada,Recurso de Amparo,Con lugar,False,...,1,[Luis Carlos Montero Benavides],None,[Director Nacional de Pensiones],"[DNP-RE-2888-05, Ley de la Jurisdicción Consti...",None,None,6599,1453,[1970-06-30 00:00:00]
6,76334.html,*060008210007CO* Exp: 06-000821-0007-CO Res. N...,PENSION,1.000000,060008210007CO,2006-06-08 11:45:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,0,"[Roy Alexander Retana Mora, Patrick Sandoval C...",None,"[Parálisis Cerebral, Departamento de Administr...",[Jefe del Departamento de Pensiones del Régime...,None,[Caja Costarricense de Seguro Social],8955,1599,None
7,7A3D3.html,*060097740007CO* Exp: 06-009774-0007-CO Res. N...,TRABAJO,0.669547,060097740007CO,2006-08-30 18:38:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,0,[JAVIER MURILLO TIJERINO],None,None,"[JEFE DEL ÁREA DE SERVICIOS, ESPECIALIZADOS DE...",None,None,4903,1315,None
8,75309.html,*060034660007CO* Exp: 06-003466-0007-CO Res. N...,PENSION,0.373961,060034660007CO,2006-04-25 15:04:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,...,0,[Luis Carlos Montero Benavides],[R-TP-DNP-NRE-0519-2005 de las 11:00],"[Director Ejecutivo, Dirección Nacional de Pen...",[Desobediencia],None,None,8654,1494,None
9,78941.html,*060045250007CO* Exp: 06-004525-0007-CO Res. N...,SALUD,0.999989,060045250007CO,2006-07-26 17:31:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,"[Manrique Soto Pacheco, Nuria Montero Chinchil...",None,[Presidente Ejecutivo de la Caja Costarricense...,[Director del Hospital San Juan de Dios],None,[Caja Costarricense de Seguro Social],15596,1844,None


# Zona de trabajo (ignorar)

In [9]:
s.filter(s.se_ordena_Ent_Pub.isNotNull()).limit(10).toPandas()
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,0,"[Marjorie Carmona Solano, Elías Villegas Matar...","[Circuito 02 de la, Colegio Líder INVU Las Cañas]","[Directora de la Escuela, Dirección Regional d...","[Director Regional de Educación de Alajuela, I]",None,"[Instituto Nacional de Vivienda y Urbanismo, I...",24112,2582,None
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,1,"[Hazel Córdoba Soto, Jefe del Área de Servicio...","[Ministerio de Educación Pública, Ministro de ...",None,None,None,[Ministerio de Educacion Publica],6022,1394,[1970-01-03 00:00:00]
2,76334.html,*060008210007CO* Exp: 06-000821-0007-CO Res. N...,PENSION,1.000000,060008210007CO,2006-06-08 11:45:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,0,"[Roy Alexander Retana Mora, Patrick Sandoval C...",None,"[Parálisis Cerebral, Departamento de Administr...",[Jefe del Departamento de Pensiones del Régime...,None,[Caja Costarricense de Seguro Social],8955,1599,None
3,78941.html,*060045250007CO* Exp: 06-004525-0007-CO Res. N...,SALUD,0.999989,060045250007CO,2006-07-26 17:31:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,"[Manrique Soto Pacheco, Nuria Montero Chinchil...",None,[Presidente Ejecutivo de la Caja Costarricense...,[Director del Hospital San Juan de Dios],None,[Caja Costarricense de Seguro Social],15596,1844,None
4,785B2.html,*060060250007CO* Exp: 06-006025-0007-CO Res. N...,TRABAJO,0.401109,060060250007CO,2006-07-04 18:24:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,...,1,[Luis Bernardo Arguedas Álvarez],None,[Coordinador del Nivel de Atención Institucion...,[Ley de la Jurisdicción Constitucional],None,[Direccion General de Adaptacion Social],6883,1386,[1970-01-09 00:00:00]
5,79926.html,*060086920007CO* Exp: 06-008692-0007-CO Res. N...,TRABAJO,0.672740,060086920007CO,2006-08-18 10:19:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,1,"[Deily Juárez Castillo, Subjefe del Área de Se...","[Ministerio de Educación Pública, la Ley]",[DIAS],None,None,[Ministerio de Educacion Publica],7708,1526,[1970-01-09 00:00:00]
6,784CC.html,*060032400007CO* Exp: 06-003240-0007-CO Res. N...,TRABAJO,0.620842,060032400007CO,2006-04-26 14:32:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,0,"[Soledad Chinchilla Arguedas, Subdirectora Gen...",[Ministerio de Educación Pública],[Consulta Externa del Hospital La Anexión de l...,None,None,"[Ministerio de Educacion Publica, Caja Costarr...",6577,1393,None
7,7599B.html,*060019500007CO* Exp: 06-001950-0007-CO Res. N...,TRABAJO,0.694684,060019500007CO,2006-05-19 16:07:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,...,0,"[Manuel Antonio Bolaños Salas, William Cordero...","[Ministro de Educación Pública, Ministerio SUS...","[DIEZ, DIAS, MAS, HASTA LA FECHA DE NOTIFICACI...",[Director General de Personal del Ministerio d...,None,[Ministerio de Educacion Publica],13078,1748,None
8,7AACC.html,*060091570007CO* Exp: 06-009157-0007-CO Res. N...,TRABAJO,0.618674,060091570007CO,NaT,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,1,"[Hazel Córdoba Soto, Jefe del Área de Servicio...",[Ministerio de Educación Pública],None,[DIEZ DÍAS],None,[Ministerio de Educacion Publica],5406,1642,None
9,75331.html,*060002950007CO* Exp: 06-000295-0007-CO Res. N...,TRABAJO,0.507462,060002950007CO,2006-04-28 12:36:00,Saborío Soto,Recurso de Amparo,Con lugar,False,...,1,"[Manuel Antonio Bolaños Salas, Hazel Córdoba S...",[Ministerio de Educación Pública],[Ministro de Educación Pública y Jefa del Área...,[Ley d

In [14]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
df2 = (s
 .where(s.se_ordena_ORG.isNotNull())
 .select(explode('se_ordena_LOC').alias('se_ordena_LOC'), 'anno')
 .groupby('se_ordena_LOC')
 .count()
 .sort(F.desc('count'))
).limit(60).toPandas()
df2

,se_ordena_LOC,count
0,Ministerio de Educación Pública,60
1,Hospital Dr. Rafael Ángel Calderón Guardia,18
2,Alajuela,13
3,Hospital San Rafael de,13
4,Sala,11
5,Ministerio de Trabajo y Seguridad,10
6,Directora del Área de Carrera Docente,9
7,Directora,7
8,Hospital Dr. Rafael Calderón Guardia,7
9,Hospital México,6


In [11]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
df2 = (s
 .where(s.se_ordena_ORG.isNotNull())
 .select(explode('se_ordena_Ent_Pub').alias('se_ordena_Ent_Pub'), 'anno')
 .groupby('se_ordena_Ent_Pub')
 .count()
 .sort(F.desc('count'))
).toPandas()
df2

,se_ordena_Ent_Pub,count
0,Ministerio de Educacion Publica,75
1,Caja Costarricense de Seguro Social,70
2,Direccion General Del Servicio Civil,16
3,Instituto de Desarrollo Agrario,5
4,Universidad Nacional,5
5,Instituto Mixto de Ayuda Social,4
6,Tribunal de Carrera Docente,3
7,Ministerio de Seguridad Publica,3
8,Instituto Costarricense de Electricidad,3
9,Patronato Nacional de La Infancia,3
